In [ ]:
!pip install -q langgraph==0.2.35 langchain_experimental==0.0.65 langchain-openai==0.1.25 termcolor==2.3.0 duckduckgo_search==7.1.0 openapi-python-client==0.12.3 langchain_community==0.2.19 wikipedia==1.4.0

In [5]:
import os
from typing import Annotated, TypedDict

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# Environment Variables
INFERENCE_SERVER_URL = os.getenv("API_URL_GRANITE")
API_KEY = os.getenv("API_KEY_GRANITE")
MODEL_NAME = "granite-3-8b-instruct"

# Initialize LLM
llm = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base=f"{INFERENCE_SERVER_URL}/v1",
    model_name=MODEL_NAME,
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True
)

# Initialize Tools
tools = [DuckDuckGoSearchRun()]
llm_with_tools = llm.bind_tools(tools)

# Define State
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Build Graph
graph_builder = StateGraph(State)

def chatbot(state: State):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": state["messages"] + [response]}  # Append response to message history

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools))
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

graph = graph_builder.compile(checkpointer=MemorySaver())
config = {"configurable": {"thread_id": "1"}}

# Agent Function
def react_agent(user_input):
    events = graph.stream({"messages": [(HumanMessage(content=user_input))]}, config, stream_mode="values")

    for event in events:
        messages = event["messages"]

        # Get only the last AI response (avoiding repeated prints)
        last_message = messages[-1]
        if isinstance(last_message, AIMessage):  # Ensure it's the LLM response
            print("\n===== \033[1mAgent Response\033[0m =====\n", last_message.content, "\n")

# Example Usage
react_agent("How many seconds would it take for a leopard at full speed to run through Pont des Arts?")



===== Agent Response =====
 <tool_call> 


===== Agent Response =====
 The Pont des Arts is a bridge in Paris, France. Its length is approximately 330 meters. A leopard can run at a top speed of about 36 miles per hour (58 km/h). 

To calculate the time it would take for a leopard to run through the bridge, we need to convert the bridge's length into feet and then into miles, as the leopard's speed is given in miles per hour. 

1 meter = 3.28084 feet
330 meters = 1079.46 feet

Now, converting feet to miles:
1 mile = 5280 feet
1079.46 feet = 0.2045 miles

Using the formula Time = Distance / Speed, we get:
Time = 0.2045 miles / 36 mph = 0.00568 hours

To convert this into seconds:
0.00568 hours * 3600 seconds/hour = 20.448 seconds

So, it would take approximately 20.45 seconds for a leopard running at full speed to cross the Pont des Arts. 

